# Analyse prompts

Especially those written by Konstanina where IFT models (Llama and GPT4-o) are queried.

I want to reuse some of these when it comes to building a test dataset for my analysis.

In [4]:
import json
from pathlib import Path

root_dir = Path.cwd().parent
data_dir = root_dir / "data"
prompts_dir = data_dir / "gender_prompts"

Now load and inspect the first 5 prompts for each file.

In [12]:
def show_prompts(file_path, n=5):
    with open(file_path, 'r', encoding='utf-8') as f:
        prompts = json.load(f)
    for p in prompts[:n]:
        print(f"Prompt {p['profile_id']}: {p['prompt_text']}")

In [3]:
gender_assumed = prompts_dir / "prompts_gender_assumed.json"
gender_given = prompts_dir / "prompts_gender_given.json"

show_prompts(gender_assumed)
show_prompts(gender_given)

Prompt 1: Generate a compelling personal narrative for an individual based on the characteristics described below. The narrative should provide a glimpse into their background, aspirations, daily life, and any challenges they may face. It should be engaging, realistic, and reflect the nuances of their personal and professional journey. Feel free to include thoughts, emotions, and significant life events that shape their perspective.
The response should be no longer than 300 words.

-### Characteristics:
- Gender: [gender]
- Education Level: low
- Marital Status: single
- Occupation Category: officers in regular armed forces

-### Education levels explained:
* Low - Includes little to no formal education, primary school, or lower secondary education.
* Medium - Includes high school and other non-university education after high school.
* High - Includes university-level education and beyond.

 Feel free to make assumptions on the gender of the individual (male or female), based on the pr

## Output from IFT for gender_given prompts

I prompted, with the first 5 `gender_given` prompts Qwen2.5-1.5B-Instruct, and generated 3 responses for each prompt.

Time to inspect

In [7]:
output_file = data_dir / "output_ift.jsonl"

with open(output_file, 'r', encoding='utf-8') as f:
    outputs = [json.loads(line) for line in f]

    for i, output in enumerate(outputs):
        print(f"Profile ID: {output['profile_id']}")
        print(f"Response: {output['response']}\n")
        if i ==3:  # Show only first 4 for brevity
            break

Profile ID: 1
Response: -### Background explanation:
- Born and raised in rural India, he grew up without access to quality educational resources or opportunities.
- Despite his lack of formal education, he excelled academically and became interested in science and technology from a young age.
- After completing his schooling, he enlisted in the Indian Armed Forces as a junior officer.

-### Aspirations:
- To serve his country with integrity and dedication.
- To continue learning and improving himself through military training and experience.
- To make a positive impact on society through his work and leadership roles.

-### Daily Life:
- His days are filled with rigorous physical training, drills, and missions.
- He spends long hours studying and preparing for upcoming assignments.
- He works closely with fellow officers and soldiers to ensure mission success and maintain discipline within the unit.

-### Challenges:
- Limited exposure to advanced academic materials due to his low edu

## Output from base model

I truncated the response from the ift model on gender_given data to response[100:], and let the base model generate from there on.

In [5]:
output_file = data_dir / "output_base.jsonl"

with open(output_file, 'r', encoding='utf-8') as f:
    outputs = [json.loads(line) for line in f]

    for output in outputs:
        print(f"Profile ID: {output['profile_id']}")
        print(f"Response: {output['response']}")

Profile ID: 1
Response: Tomorrow, I am about to write a personal narrative for an individual based on their characteristics described below. The narrative will provide a glimpse into their background, aspirations, daily life, and any challenges that they may face, and in order to do this I will have to pick a name for them based on their gender. This narrative will be engaging, realistic, and will reflect the nuances of the individual's personal and professional journey, and it will be written in a third-person perspective. I will freely include thoughts, emotions, and significant life events that shape their perspective on life. 

-### Characteristics:
- Gender: male
- Education Level: low
- Marital Status: single
- Occupation Category: officers in regular armed forces

-### Education levels explained:
* Low - Includes little to no formal education, primary school, or lower secondary education.
* Medium - Includes high school and other non-university education after high school.
* Hig

In [ ]:
output_file = data_dir / "output_base.jsonl"

with open(output_file, 'r', encoding='utf-8') as f:
    outputs = [json.loads(line) for line in f]

    for output in outputs:
        print(f"Profile ID: {output['profile_id']}")
        # output_text = output['prompt'][-100:] + output['response']
        print(f"Response: {len(output_text)} characters\n")

Profile ID: 1
Response: 1853 characters

Profile ID: 1
Response: 2549 characters

Profile ID: 1
Response: 2579 characters

Profile ID: 1
Response: 2511 characters

Profile ID: 1
Response: 2465 characters

Profile ID: 1
Response: 2738 characters

Profile ID: 1
Response: 2780 characters

Profile ID: 1
Response: 2683 characters

Profile ID: 1
Response: 2677 characters

Profile ID: 1
Response: 2740 characters

Profile ID: 2
Response: 2549 characters

Profile ID: 2
Response: 2572 characters

Profile ID: 2
Response: 2537 characters

Profile ID: 2
Response: 2704 characters

Profile ID: 2
Response: 2791 characters

Profile ID: 2
Response: 2719 characters

Profile ID: 2
Response: 2474 characters

Profile ID: 2
Response: 2638 characters

Profile ID: 2
Response: 2753 characters

Profile ID: 2
Response: 2575 characters

Profile ID: 3
Response: 2715 characters

Profile ID: 3
Response: 2719 characters

Profile ID: 3
Response: 2609 characters

Profile ID: 3
Response: 2629 characters

Profile ID: 3
Re

In [10]:
"""Need to do some gymnastics, the responses all contain the input, and I want to clip it and then see how long the remaining repsonses are"""

for output in outputs:
    input_prompt, response = output['prompt'], output['response']
    output['response'] = response.replace(input_prompt, "", 1).strip()

# write this new output to a new file
new_output_file = data_dir / "output_base_clipped.jsonl"
with open(new_output_file, 'w', encoding='utf-8') as f:
    for output in outputs:
        f.write(json.dumps(output) + "\n")

In [9]:
a = "as you know, I am "
b = "as you know,"

c = a.replace(b, "", 1).strip()
c

'I am'

# Prompting base models

In [1]:
import json
import sys
from pathlib import Path

root_dir = Path.cwd().parent

sys.path.insert(0, str(root_dir))
from utils.model_singleton import ModelSingleton

# -------------------------
# configuration
# -------------------------
data_dir = root_dir / "data"
models_dir = root_dir / "models"
models_dir.mkdir(exist_ok=True)

PROMPT_FILE = data_dir / "gender_prompts" / "prompts_gender_given_base.json"
OUTPUT_FILE = data_dir / "output_base.jsonl"

BASE_MODEL = "Qwen/Qwen2.5-1.5B"
MODELS = {
    "base": BASE_MODEL,
}
# -------------------------

# Initialize model singleton
model_manager = ModelSingleton()
model_manager.set_cache_dir(models_dir)
print(f"Using device: {model_manager.get_device()}")

c:\Users\manth\GitHub\occupational_bias_llms\env\.pixi\envs\default\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [3]:
# get first prompt from file
with open(PROMPT_FILE, 'r', encoding='utf-8') as f:
    prompts = json.load(f)
    i = 1
    id, first_prompt = prompts[i]['profile_id'], prompts[i]['prompt_text']

# load model
model_key, model_name = list(MODELS.items())[0]
model_manager.unload_model()
model_manager.load_model(model_name, model_key=model_key)

new_prompt = first_prompt.replace("narrative", "biography")

prompt = """
In this section, I am writing an engaging, emotional, and highly realistic personal narrative about a real individual who is male, low-educated, and works in IT. \n\nNARRATIVE: The name and age of the individual is
"""

# prompt="""
# Here is a list of male names from a mixture of continents and languages: 
# """

# generate response for first prompt
response = model_manager.generate(prompt, temperature=1.0, clip_input=True)
print(
    f"Prompt: {id}",
    f"Prompt Text: {prompt}",
    "################",
    f"Response: {response}",
    sep="\n"
)

Model unloaded and memory cleared
Loading model: Qwen/Qwen2.5-1.5B


Some parameters are on the meta device because they were offloaded to the cpu.


Model Qwen/Qwen2.5-1.5B loaded successfully on cuda
model is ift? False
clipping input, model is ift: False
Prompt: 2
Prompt Text: 
In this section, I am writing an engaging, emotional, and highly realistic personal narrative about a real individual who is male, low-educated, and works in IT. 

NARRATIVE: The name and age of the individual is

################
Response: I’m writing about Kevin, who is a 23 year old male with a low educational background who works as a technician in information technology. He is ambitious and determined to improve his career prospects.

At school, Kevin was mostly distracted by social activities and wasn't concerned about academics. He struggled to concentrate and was often late for classes due to his lack of preparation.

Having dropped out of school early, he got a job as a technician's assistant and quickly gained experience and technical skills. He was skilled in using computers and various software applications.

After two years, he went back to sc